<a href="https://colab.research.google.com/github/Nasim-Mahmud/LSTM-projects/blob/main/lstm_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
paths = "/content/test_data.docs"
docs = ""
with open(paths, 'r') as doc:
  docs = doc.read()
docs

'Human nature, a concept as old as time itself, has captivated the minds of philosophers, scientists, and ordinary individuals alike. It is a complex and multifaceted enigma, a tapestry woven from threads of biology, psychology, sociology, and culture. Understanding human nature is not merely an academic pursuit; it is a fundamental quest to comprehend the very essence of our existence, the driving forces that shape our behaviors, and the potential that lies dormant within us. From a biological perspective, human nature is deeply rooted in our evolutionary heritage. We are social creatures, driven by an innate need for connection and belonging. Our brains, the most complex organs on Earth, possess remarkable capacities for learning, reasoning, and creativity. We are also emotional beings, capable of experiencing a vast spectrum of feelings, from love and joy to fear and anger. These biological underpinnings provide the foundation upon which human nature is constructed. Psychology delve

In [359]:
# Need to build a dataset based on the given text, so first we need to convert this strings into integers so that our model can work on it.
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [360]:
tokenizer = Tokenizer()

In [361]:
# Assigning a integer to every number
tokenizer.fit_on_texts([docs])
# See the tokenized words
# tokenizer.word_index

In [362]:
input_seq = []
# Separating all the sentences based on line break.\
for sentence in docs.split("\n"):
  # Converting every sentence to a sequence using its designated integer values. Getting a numeric representation.
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  # Converting sequences to a dataset of given input and output
  for i in range(1, len(tokenized_sentence)):
    input_seq.append(tokenized_sentence[:i+1])

In [363]:
# input_seq

In [364]:
lengthOfLargestSeq = max([len(x) for x in input_seq])
lengthOfLargestSeq

488

In [365]:
# Using zero padding to make every sequence equal in length, Sparse vectorization.
from keras.preprocessing.sequence import pad_sequences
padded_input_seq = pad_sequences(input_seq, maxlen = lengthOfLargestSeq, padding= 'pre')
padded_input_seq

array([[  0,   0,   0, ...,   0,   6,   7],
       [  0,   0,   0, ...,   6,   7,   5],
       [  0,   0,   0, ...,   7,   5,  31],
       ...,
       [  0,   0,   6, ..., 240,   2, 241],
       [  0,   6,   7, ...,   2, 241,   3],
       [  6,   7,   5, ..., 241,   3, 242]], dtype=int32)

In [366]:
X = padded_input_seq[:, :-1]
X

array([[  0,   0,   0, ...,   0,   0,   6],
       [  0,   0,   0, ...,   0,   6,   7],
       [  0,   0,   0, ...,   6,   7,   5],
       ...,
       [  0,   0,   6, ..., 239, 240,   2],
       [  0,   6,   7, ..., 240,   2, 241],
       [  6,   7,   5, ...,   2, 241,   3]], dtype=int32)

In [367]:
y = padded_input_seq[:, -1]

In [368]:
X.shape

(487, 487)

In [369]:
y.shape

(487,)

In [370]:
numOfWords = len(tokenizer.word_index)

In [371]:
# Using one hot encoding
from keras.utils import to_categorical
y = to_categorical(y, num_classes= numOfWords + 1)
y.shape

(487, 243)

# **Building the LSTM architecture**

In [372]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [373]:
model = Sequential()

# Converting Sparse vectorization to Embedding, shape will be 100, input lenght will be the length of largest sequence -1.
model.add(Embedding(numOfWords + 1, 100, input_length= lengthOfLargestSeq - 1))

# Number of nodes in the gate of LSTM layer
model.add(LSTM(150))
model.add(Dense(numOfWords + 1, activation= "softmax"))

In [374]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [375]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 487, 100)          24300     
                                                                 
 lstm_7 (LSTM)               (None, 150)               150600    
                                                                 
 dense_7 (Dense)             (None, 243)               36693     
                                                                 
Total params: 211593 (826.54 KB)
Trainable params: 211593 (826.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [376]:
model.fit(X, y, epochs= 100)

Epoch 1/100
16/16 [==============================] - 16s 874ms/step - loss: 5.4486 - accuracy: 0.0513
Epoch 2/100
16/16 [==============================] - 14s 865ms/step - loss: 5.0862 - accuracy: 0.0637
Epoch 3/100
16/16 [==============================] - 14s 852ms/step - loss: 4.9308 - accuracy: 0.0370
Epoch 4/100
16/16 [==============================] - 13s 847ms/step - loss: 4.8973 - accuracy: 0.0637
Epoch 5/100
16/16 [==============================] - 14s 859ms/step - loss: 4.8886 - accuracy: 0.0534
Epoch 6/100
16/16 [==============================] - 13s 825ms/step - loss: 4.8632 - accuracy: 0.0657
Epoch 7/100
16/16 [==============================] - 13s 786ms/step - loss: 4.8458 - accuracy: 0.0637
Epoch 8/100
16/16 [==============================] - 13s 769ms/step - loss: 4.8229 - accuracy: 0.0821
Epoch 9/100
16/16 [==============================] - 13s 790ms/step - loss: 4.7753 - accuracy: 0.0821
Epoch 10/100
16/16 [==============================] - 14s 826ms/step - loss: 4.687

In [377]:
import numpy as np

In [378]:
def word_predictor(numOfWordsToGenerate, text):
  for i in range(numOfWordsToGenerate):
    #tokenier
    tokenized_text = tokenizer.texts_to_sequences([text])[0]
    #padding
    padded_text = pad_sequences([tokenized_text], maxlen= lengthOfLargestSeq - 1, padding= "pre")
    #predict
    position = np.argmax(model.predict(padded_text, verbose=0))

    for word, index in tokenizer.word_index.items():
      if index == position:
        text = text + " " + word
  print(text)

In [379]:
num_of_words_to_generate = 20
given_words = "What is the"
word_predictor(num_of_words_to_generate, given_words)

What is the nature a concept as old as time itself has captivated the minds of philosophers scientists and ordinary individuals alike it
